In [22]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from skimage import data

# --- 1) Charger un chiffre (8x8) depuis skimage ---
digits = data.digits()          # shape (1797, 8, 8), valeurs 0..16
idx = 0
img8 = digits.images[idx].astype(np.float32)  # 0..16

# Agrandir en 28x28 pour "imiter" MNIST
img = cv2.resize(img8, (28, 28), interpolation=cv2.INTER_NEAREST)

# Normaliser -> uint8 0..255 (image "originale")
img_u8 = (img / img.max() * 255).astype(np.uint8)

# --- 2) Créer une carte de probabilité artificielle (bruit + flou) ---
prob = img_u8.astype(np.float32) / 255.0
noise = np.random.normal(0.0, 0.15, size=prob.shape).astype(np.float32)
prob_noisy = np.clip(prob + noise, 0.0, 1.0)
prob_blur = cv2.GaussianBlur(prob_noisy, (3, 3), 0)

# Repasser en "image proba" 0..255 (comme ton dossier probabilities/)
prob_img_255 = (prob_blur * 255).astype(np.uint8)

# --- 3) Post-processing (ton pipeline) ---
processed = post_process_single_mask(
    prob_or_mask=prob_img_255,
    is_prob_map=True,
    threshold=0.5,
    morph_op="open",
    morph_kernel=3,
    morph_iter=1,
    min_size=20,        # MNIST-like petit => min_size faible
    merge_distance=1
)

# --- 4) Affichage Avant / Après ---
plt.figure(figsize=(12, 4))

plt.subplot(1, 4, 1)
plt.title("Image (digits)\n0..255")
plt.imshow(img_u8, cmap="gray")
plt.axis("off")

plt.subplot(1, 4, 2)
plt.title("Prob map (noisy+blur)\n0..1")
plt.imshow(prob_blur, cmap="gray")
plt.axis("off")

plt.subplot(1, 4, 3)
plt.title("Prob map en image\n0..255")
plt.imshow(prob_img_255, cmap="gray")
plt.axis("off")

plt.subplot(1, 4, 4)
plt.title("Après post-processing\nbinaire 0/255")
plt.imshow(processed, cmap="gray")
plt.axis("off")

plt.show()


AttributeError: No skimage.data attribute digits

In [23]:
import numpy as np
import matplotlib.pyplot as plt
import cv2

# Image noire 28x28
img_u8 = np.zeros((28, 28), dtype=np.uint8)

# Ecrire un chiffre (ex: 5)
cv2.putText(img_u8, "5", (5, 23), cv2.FONT_HERSHEY_SIMPLEX, 0.9, 255, 2, cv2.LINE_AA)

# Prob map artificielle
prob = img_u8.astype(np.float32) / 255.0
noise = np.random.normal(0.0, 0.15, size=prob.shape).astype(np.float32)
prob_blur = cv2.GaussianBlur(np.clip(prob + noise, 0, 1), (3, 3), 0)
prob_img_255 = (prob_blur * 255).astype(np.uint8)

processed = post_process_single_mask(
    prob_or_mask=prob_img_255,
    is_prob_map=True,
    threshold=0.5,
    morph_op="open",
    morph_kernel=3,
    morph_iter=1,
    min_size=20,
    merge_distance=1
)

plt.figure(figsize=(10,3))
plt.subplot(1,3,1); plt.title("Original"); plt.imshow(img_u8, cmap="gray"); plt.axis("off")
plt.subplot(1,3,2); plt.title("Prob map"); plt.imshow(prob_img_255, cmap="gray"); plt.axis("off")
plt.subplot(1,3,3); plt.title("Après"); plt.imshow(processed, cmap="gray"); plt.axis("off")
plt.show()


error: OpenCV(4.12.0) :-1: error: (-5:Bad argument) in function 'putText'
> Overload resolution failed:
>  - img is not a numpy array, neither a scalar
>  - Expected Ptr<cv::UMat> for argument 'img'


In [24]:
import numpy as np
import matplotlib.pyplot as plt
import cv2

# 1) Image noire 28x28 + chiffre
img_u8 = np.zeros((28, 28), dtype=np.uint8)
cv2.putText(img_u8, "5", (5, 23), cv2.FONT_HERSHEY_SIMPLEX, 0.9, 255, 2, cv2.LINE_AA)

# 2) Simuler une "prob map" : bruit + flou + artefacts
prob = img_u8.astype(np.float32) / 255.0

# bruit gaussien
noise = np.random.normal(0.0, 0.20, size=prob.shape).astype(np.float32)
prob_noisy = np.clip(prob + noise, 0, 1)

# bruit sel/poivre (petits points)
sp = np.random.rand(*prob.shape)
prob_noisy[sp < 0.03] = 1.0   # sel
prob_noisy[sp > 0.97] = 0.0   # poivre

# petit "trou" (simulateur d'erreur de segmentation)
prob_noisy[14:16, 12:16] *= 0.0

# flou pour bords incertains
prob_blur = cv2.GaussianBlur(prob_noisy, (3, 3), 0)

# proba en image 0..255 (comme ton dossier probabilities/)
prob_img_255 = (prob_blur * 255).astype(np.uint8)

# 3) Post-processing
processed = post_process_single_mask(
    prob_or_mask=prob_img_255,
    is_prob_map=True,
    threshold=0.5,
    morph_op="open",     # enlève le bruit
    morph_kernel=3,
    morph_iter=1,
    min_size=20,         # petit car image 28x28
    merge_distance=1
)

# (option) Essaye aussi "close" pour combler le trou
processed_close = post_process_single_mask(
    prob_or_mask=prob_img_255,
    is_prob_map=True,
    threshold=0.5,
    morph_op="close",    # comble trous
    morph_kernel=3,
    morph_iter=1,
    min_size=20,
    merge_distance=1
)

# 4) Affichage
plt.figure(figsize=(12, 3))

plt.subplot(1, 4, 1)
plt.title("Original (chiffre)")
plt.imshow(img_u8, cmap="gray")
plt.axis("off")

plt.subplot(1, 4, 2)
plt.title("Prob map (bruit+flou)")
plt.imshow(prob_img_255, cmap="gray")
plt.axis("off")

plt.subplot(1, 4, 3)
plt.title("Après (OPEN)")
plt.imshow(processed, cmap="gray")
plt.axis("off")

plt.subplot(1, 4, 4)
plt.title("Après (CLOSE)")
plt.imshow(processed_close, cmap="gray")
plt.axis("off")

plt.show()


error: OpenCV(4.12.0) :-1: error: (-5:Bad argument) in function 'putText'
> Overload resolution failed:
>  - img is not a numpy array, neither a scalar
>  - Expected Ptr<cv::UMat> for argument 'img'
